In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from castep_parse import read_cell_file, read_geom_file, read_castep_file
import numpy as np

from utilities import decompress_file

In [3]:
def verify_castep_cell_with_atomistic_structure(cell_file, structure):
    """Verify the atoms and supercell listed in a CASTEP cell file are
    equivalent to those of an AtomisticStructure."""
    
    cst_cell = read_cell_file(cell_file)
        
    # Supercell:
    cst_supercell = cst_cell['supercell']
    struct_supercell = structure.supercell    
    assert np.allclose(cst_supercell, struct_supercell)
    
    # Atoms:
    struct_atoms_srt_idx = np.lexsort(np.round(structure.atoms.coords, decimals=9))
    struct_atoms = structure.atoms.coords[:, struct_atoms_srt_idx]
    
    
    cst_atoms_srt_idx = np.lexsort(np.round(cst_cell['atom_sites'], decimals=9))
    cst_atoms = cst_cell['atom_sites'][:, cst_atoms_srt_idx]
    
    # Wrap atoms to within the supercell:    
    struct_atoms_sup = np.linalg.inv(struct_supercell) @ struct_atoms
    cst_atoms_sup = np.linalg.inv(cst_supercell) @ cst_atoms
    
    boundary_idx = structure.boundary_idx
    struct_atoms_sup[boundary_idx] -= np.floor(struct_atoms_sup[boundary_idx])
    cst_atoms_sup[boundary_idx] -= np.floor(cst_atoms_sup[boundary_idx])
    
    bad_atoms_idx = np.where(np.abs(cst_atoms_sup - struct_atoms_sup) > 1e-6)[1]
    
    assert np.allclose(struct_atoms_sup, cst_atoms_sup)


## Ab-initio tensile tests

In [3]:
from tensile_test import get_tensile_test, show_traction_separation

**s7-tw-b-a1**

In [4]:
tt = get_tensile_test('s7-tw-b-a1')
test_input_structure = tt.get_coordinate(2.1).structure

Geometry optimisation finished in run 0


In [5]:
cell_path_compressed = Path('data/ab_initio_tensile_tests/rgs-s7-tw-b-a1/16_+2.100/sim.cell.xz')
cell_file = decompress_file(cell_path_compressed)

In [6]:
verify_castep_cell_with_atomistic_structure(cell_file, test_input_structure)

**s7-tlA-gb-a1**

In [7]:
tt = get_tensile_test('s7-tlA-gb-a1')
test_input_structure = tt.get_coordinate(1.5).structure

Geometry optimisation finished in run 1


In [8]:
cell_path_compressed = Path('data/ab_initio_tensile_tests/rgs-s7-tlA-gb-a1/03_+1.500/sim.cell.xz')
cell_file = decompress_file(cell_path_compressed)

In [9]:
verify_castep_cell_with_atomistic_structure(cell_file, test_input_structure)


## Compare `make_structure` and `get_simulated_bicrystal`

In [6]:
from utilities import make_structure, get_simulated_bicrystal, get_simulation

In [7]:
structure_code = 's7-tlA-fs'
bicrystal_1 = make_structure(structure_code, 'minimum_energy')
bicrystal_2 = get_simulated_bicrystal(structure_code, opt_idx=0)
bicrystal_1.same_atoms(bicrystal_2)

Geometry optimisation finished in run 0
supercells are the same.


True

## Compare structure in `get_simulated_bicrystal` and `get_simulation`

In [4]:
from utilities import make_structure, get_simulated_bicrystal, get_simulation

### Initial

In [10]:
structure_code = 's7-tlA-b'

In [11]:
opt_idx = -1

structure = get_simulated_bicrystal(structure_code, opt_idx=opt_idx)
atoms_1 = structure.atoms.coords
srt_idx_1 = np.lexsort(np.round(atoms_1, decimals=8))
atoms_1_srt = atoms_1[:, srt_idx_1]

# geom_dat = read_geom_file(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\defects\b-a1\sim.geom")
# atoms_2 = geom_dat['atoms'][opt_idx]
# srt_idx_2 = np.lexsort(np.round(atoms_2, decimals=8))
# atoms_2_srt = atoms_2[:, srt_idx_2]

# np.allclose(atoms_1_srt, atoms_2_srt)

Geometry optimisation finished in run 0
get_simulated_bicrystal: atoms same?: True


In [12]:
gb_sim = get_simulation(structure_code)
sim_structure = gb_sim.get_step(opt_step=opt_idx)['structure']

atoms_3 = sim_structure.atoms.coords
srt_idx_3 = np.lexsort(np.round(atoms_3, decimals=8))
atoms_3_srt = atoms_3[:, srt_idx_3]

Geometry optimisation finished in run 0
A get_simulation: atoms[-1, :, 41] 
[ 1.2910227  32.34947686 44.05115531]



In [13]:
gb_sim = get_simulation(structure_code)
sim_structure = gb_sim.get_step(opt_step=opt_idx)['structure']

atoms_3 = sim_structure.atoms.coords
srt_idx_3 = np.lexsort(np.round(atoms_3, decimals=8))
atoms_3_srt = atoms_3[:, srt_idx_3]

np.allclose(atoms_1_srt, atoms_3_srt)

Geometry optimisation finished in run 0
A get_simulation: atoms[-1, :, 41] 
[ 1.2910227  32.34947686 44.05115531]



True

### Final

In [18]:
from castep_parse import read_castep_file, read_geom_file, read_cell_file, merge_output_data

In [21]:
opt_idx = 0

cp = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\defects\b-a1\sim.castep")
gp = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\defects\b-a1\sim.geom")
ce = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\defects\b-a1\sim.cell")

# cp = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\sim.castep")
# gp = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\sim.geom")
# ce = Path(r"D:\phd\calc_dirs_2018.11.29\calcs\calcs\2018-01-30-1632_67391\calcs\1_1_3\sim.cell")

cst_dat = read_castep_file(cp)
geom_dat = read_geom_file(gp)
cell_dat = read_cell_file(ce)
cstp_dat = merge_output_data(cst_dat, geom_dat=geom_dat, cell_dat=cell_dat)
atoms_1 = cstp_dat['geom']['iterations'][opt_idx]['atoms']

sim_2 = get_simulation(structure_code)
sim_step_2 = sim_2.get_step(opt_step=opt_idx, atom_site_geometries=False)
atoms_2 = sim_step_2['structure'].atoms.coords

np.allclose(atoms_1, atoms_2)

Geometry optimisation finished in run 1
Geometry optimisation finished in run 1
no atoms in geom iter!
structure reversion?


False

In [78]:
atom_idx = 41

In [58]:
(sim_2.structure.atoms.coords + sim_2.atom_displacements[-1])[:, atom_idx]

array([ 1.28867788, 32.56071929, 44.05664453])

In [59]:
atoms_2[:, atom_idx]

array([ 1.28867788, 32.56071929, 44.05664453])

In [60]:
atoms_1[:, atom_idx]

array([3.8707313 , 2.04593796, 0.36083756])

In [61]:
final_atom_AU = np.array([
    [7.3121919981917589E+000, 2.7602336143738846E+001, 4.2649310779223292E+001],
    [7.3070887718526603E+000, 3.2201564255190206E+001, 4.6627138768103784E+001],
    [7.3174387012991442E+000, 3.7990706212907121E+001, 4.4631992979326476E+001],
    [7.3068301719705504E+000, 3.1074598300718101E+001, 5.2621603068778882E+001],
    [7.2916177845992065E+000, 3.6836529343610088E+001, 5.0631544228824659E+001],
    [7.1846512744416406E+000, 4.2576287895336286E+001, 4.8652052165169529E+001],
    [7.2740128711624026E+000, 3.5701099595200745E+001, 5.6650264992342024E+001],
    [7.2561535473942556E+000, 4.1449593627602297E+001, 5.4637475024757876E+001],
    [7.1806883945928197E+000, 4.0355522902068650E+001, 6.0619053561749695E+001],
    [7.3104340976418110E+000, 4.6086130968004845E+001, 5.8622506634841670E+001],
    [7.3023536070426092E+000, 3.9167055302196019E+001, 6.6613119691817943E+001],
    [7.2877014092796761E+000, 4.4931395583016950E+001, 6.4626525152687279E+001],
    [7.1818656386711464E+000, 5.0660710498929376E+001, 6.2640768714494889E+001],
    [7.2825108204911801E+000, 4.3802135597062048E+001, 7.0613943763795547E+001],
    [7.2558888528821264E+000, 4.9547289728903223E+001, 6.8655136391538477E+001],
    [7.2923911911546710E+000, 4.8454758847401706E+001, 7.4739383396263776E+001],
    [7.3167366959489675E+000, 5.4187161829824582E+001, 7.2606904677368476E+001],
    [7.3198710115300729E+000, 4.7487250677893400E+001, 8.0606513035828769E+001],
    [7.3197283089431959E+000, 5.3286091919484356E+001, 7.8724180898201027E+001],
    [7.3218105375301903E+000, 5.8850043556892771E+001, 7.6515396593465667E+001],
    [7.3200583142238038E+000, 5.8005144066304318E+001, 8.2643596645984545E+001],
    [2.4454044328924840E+000, 3.1105566409286102E+001, 4.3276425116807779E+001],
    [2.4100778201450925E+000, 2.9951108378174670E+001, 4.9297659221216634E+001],
    [2.4385139402384075E+000, 3.5718169590462139E+001, 4.7308316890575796E+001],
    [2.4123666397195129E+000, 4.1482631059066499E+001, 4.5297815540219204E+001],
    [2.4109328374021075E+000, 3.4589594246261953E+001, 5.3316622210781652E+001],
    [2.3010712021053163E+000, 4.0357120170273646E+001, 5.1294718228744536E+001],
    [2.3013848869239366E+000, 3.9195653462951995E+001, 5.7371861540144991E+001],
    [2.4299153587331102E+000, 4.4944991438008955E+001, 5.5299661319778764E+001],
    [2.3804488247589406E+000, 3.8045748026803679E+001, 6.3311228704468583E+001],
    [2.4044443752694278E+000, 4.3821614403028107E+001, 6.1319904651001792E+001],
    [2.4036524153709862E+000, 4.9567881949227122E+001, 5.9309241768361630E+001],
    [2.4211229685947151E+000, 4.2678125432245857E+001, 6.7324858078364770E+001],
    [2.3015514027502149E+000, 4.8438159495128247E+001, 6.5292595734846770E+001],
    [2.3100298028744648E+000, 4.7272777383094372E+001, 7.1379702820097322E+001],
    [2.4262026785639592E+000, 5.3032288610960862E+001, 6.9320085988082866E+001],
    [2.4346618913904590E+000, 4.6127437677792130E+001, 7.7373178554581060E+001],
    [2.4299638683898155E+000, 5.1929970750967342E+001, 7.5353102165965964E+001],
    [2.4279811238382996E+000, 5.7636403010765335E+001, 7.3303214395276271E+001],
    [2.4395384745470476E+000, 5.0806949667451079E+001, 8.1261485660935151E+001],
    [2.4392594685378235E+000, 5.6605716806611532E+001, 7.9373231547906130E+001],
    [7.3146220662016050E+000, 3.8662624203112075E+000, 6.8188415574493166E-001],
    [7.3514578729285018E+000, 8.1967652193926668E+000, 4.7648595741847819E+000],
    [7.3224368278855838E+000, 1.4105864300019702E+001, 2.6674113941658630E+000],
    [7.3566415512571366E+000, 6.9226730750617200E+000, 1.0666436018214299E+001],
    [7.3201749071418494E+000, 1.2745711041497884E+001, 8.5693040656887085E+000],
    [7.3489142293627321E+000, 1.8541787697374929E+001, 6.6088110653241703E+000],
    [7.3373212817967834E+000, 1.1546807064497228E+001, 1.4619106757620701E+001],
    [7.3195546726970013E+000, 1.7301706409247529E+001, 1.2638972745964894E+001],
    [7.4560776684225365E+000, 1.6136538790897170E+001, 1.8636027249314587E+001],
    [7.3584700388414515E+000, 2.1890411034847855E+001, 1.6628243696276456E+001],
    [7.3270531000960029E+000, 1.4934137567191463E+001, 2.4601287350155033E+001],
    [7.3537408227776417E+000, 2.0716510710358257E+001, 2.2597774548388394E+001],
    [7.3316183507655950E+000, 2.6503431283064607E+001, 2.0590572564545127E+001],
    [7.4551974198131070E+000, 1.9525790865596406E+001, 2.8567911589291345E+001],
    [7.3358876718640689E+000, 2.5306624067100913E+001, 2.6596292237586283E+001],
    [7.3343547524702384E+000, 2.4123796205279938E+001, 3.2595588485942770E+001],
    [7.3525183503691958E+000, 2.9922173394635998E+001, 3.0612009939044121E+001],
    [7.3224735670569094E+000, 2.2997826423820879E+001, 3.8609751891822228E+001],
    [7.3442876537062309E+000, 2.8758364254274817E+001, 3.6626898898295984E+001],
    [7.4665878720617078E+000, 3.4499659458374808E+001, 3.4649231321495428E+001],
    [7.3453957845568789E+000, 3.3372275766883064E+001, 4.0632891711279015E+001],
    [2.4413115122779163E+000, 7.0920098439872268E+000, 1.3089840860119333E+000],
    [2.5645129498477965E+000, 5.8794012141343215E+000, 7.2833164942184805E+000],
    [2.4448029055173315E+000, 1.1667974921299637E+001, 5.1895104140489243E+000],
    [2.4443565186115643E+000, 1.7469293388519944E+001, 3.2887576534990752E+000],
    [2.4525116822633959E+000, 1.0424955531533985E+001, 1.1240944627207018E+001],
    [2.4469281889791894E+000, 1.6229545262272246E+001, 9.2374979427075594E+000],
    [2.4660358530529742E+000, 1.4989734093589878E+001, 1.5244411649615881E+001],
    [2.4505455964053073E+000, 2.0764510856532503E+001, 1.3257086372802874E+001],
    [2.4734519780487059E+000, 1.3797573151004284E+001, 2.1251051297268340E+001],
    [2.5765259722155185E+000, 1.9537888759914587E+001, 1.9236662920224450E+001],
    [2.4697555957143211E+000, 2.5352718642316944E+001, 1.7247488770161102E+001],
    [2.4709593652193220E+000, 1.8424326336088782E+001, 2.5225654920183313E+001],
    [2.4595845286601508E+000, 2.4182946142857901E+001, 2.3222719188696807E+001],
    [2.4678001086243722E+000, 2.3041786991941251E+001, 2.9243015415458807E+001],
    [2.4474942171361729E+000, 2.8798086003874531E+001, 2.7262132824144707E+001],
    [2.4693639065230615E+000, 2.1883276421596850E+001, 3.5268957715473292E+001],
    [2.4578999567647877E+000, 2.7675577484741474E+001, 3.3278450348536254E+001],
    [2.5880986090390272E+000, 3.3460191126705709E+001, 3.1204991462123626E+001],
    [2.4495208515864251E+000, 2.6506229064401822E+001, 3.9281038745167997E+001],
    [2.5740974314862854E+000, 3.2285688305935835E+001, 3.7286706549599145E+001],
    [2.4419936973210850E+000, 3.6873380717903430E+001, 4.1286640003374977E+001],
    [2.4439708788716339E+000, 6.1332302189082100E+001, 8.3259209282803766E+001],    
])
final_atom_geom = final_atom_AU * 0.52917721092
final_atom_geom[atom_idx]

array([3.8707313 , 2.04593796, 0.36083756])

In [ ]:
sup = cstp_dat['geom']['iterations'][-1]['cell']
final_first_atom_frac = np.array([0.749297, 0.184446, 0.508237])
final_first_atom = sup @ final_first_atom_frac
final_first_atom

### Intermediate

In [16]:
opt_idx = 2

gb_sim = get_simulation(structure_code)
gb_sim_step = gb_sim.get_step(opt_idx)
structure_1 = gb_sim_step['structure']

structure_2 = get_simulated_bicrystal(structure_code, opt_idx=opt_idx)

structure_1.same_atoms(structure_2)

Geometry optimisation finished in run 1
Geometry optimisation finished in run 1


False

## Generate CASTEP files for testing castep-parse

In [4]:
from atomistic.bulk import BulkCrystal
from castep_parse import write_input_files

In [5]:
crystal_structure = {
    'lattice': {
        'lattice_system': 'hexagonal',
        'lattice_parameters': {
            'a': 3.2316,
            'c': 5.1475,
        },
    },
    'motif': {
        'atoms': {
            'coords': [
                [2/3, 1/3],
                [1/3, 2/3],
                [3/4, 1/4],
            ],
            'labels': {
                'species': [['Zr'], [0, 0]]
            },
        }
    },
}

In [9]:
repeats = np.eye(3) * 2
bulk = BulkCrystal.from_crystal_structure(crystal_structure, repeats)
bulk.show()

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'supercell',
              'type': 'scatte…

In [10]:
# Perturb atoms to allow a non-trivial geometry optimisation with a fixed cell
bulk.atoms += (np.random.random((3, 16)) * 0.3)

In [11]:
bulk.show()

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'supercell',
              'type': 'scatte…

In [12]:
cell_dat = {
    'supercell': bulk.supercell,
    'atom_sites': bulk.atoms.coords,
    'species': bulk.atoms.labels['species'].unique_values,
    'species_idx': bulk.atoms.labels['species'].values_idx,
}

param = {'task': 'singlepoint'}

dir_path = r'C:\code_local\scratch\_castep_sims\castep_parse_tests\base'

write_input_files(**cell_dat, dir_path=dir_path, param=param)

[WindowsPath('C:/code_local/scratch/_castep_sims/castep_parse_tests/base/sim.param'),
 WindowsPath('C:/code_local/scratch/_castep_sims/castep_parse_tests/base/sim.cell')]